# ADAPT Pro - Topic 3 - Automation, Visualization and Best Practices

# Importing Data from PDF Files

**Useful links**
- stack post on using tabula package: https://jpmc.stackenterprise.co/questions/56013
- tabula package documentaiton: https://tabula-py.readthedocs.io/en/latest/

In [24]:
import os
import tabula as tb
from tabula import read_pdf

In [25]:
tb.__version__

https://jpmc.stackenterprise.co/questions/56013

In [2]:
#Code to ensure Java is enabled for Tabula
def get_java_bin_path(version):
    from pathlib import Path
    import version_prod
    p = Path(f"/apps/asg/packages/jdkpy27-{version_prod.versions['athena']['prod']['jdk']}/g493_linux64/jdk{version}/bin")
    if not p.exists():
        raise ValueError('{p} not found')
    return p


os.environ['PATH'] = f"{get_java_bin_path('1.8.0_45')}:{os.environ['PATH']}"

## Simple Example - PDF File on Athena

In [99]:
#Need to upload a smaple pdf file before running code below
#df = read_pdf("report.pdf", pages='2')
#df.head()

## Advanced Example - PDF File on Web

On J.P. Morgan device need to use proxy server to connect to external sites

Steps:
1. create a dictionary with JPM proxies (can copy from below)
2. connect to the website where pdf is hosted using `response = requests.get(url, proxies)` and provide dictionary created to `proxies` argument
3. save the response from website with `response.content`
4. use the `io` packages and `io.BytesIO` to convert the raw content response as input to the tabula read_pdf() formula:
```
raw_pdf = response.content
df = read_pdf(io.BytesIO(raw_pdf), pages='2')
```

If scraping a pdf from a website at home, you can skips steps above and provide url to pdf directly to the read_pdf formula.

In [27]:
#Step 1 - Import packages and set up dictionary for proxies
import io
import requests
from tabula import read_pdf
proxies = { 'http' : 'http://proxy.jpmchase.net:8443/',
           'https' : 'http://proxy.jpmchase.net:8443/' }

In [28]:
#Step 2 - Connect to website
url = 'https://knowthefactsmmj.com/wp-content/uploads/ommu_updates/2023/042123-OMMU-Update.pdf'
response = requests.get(url, proxies=proxies)

In [29]:
#Step 3 - Save response from website
raw_pdf = response.content
#response.content

In [30]:
#Step 4 - Convert raw content to table with tabula read_pdf and io package
df = read_pdf(io.BytesIO(raw_pdf), pages='2')
df.head()

## Advanced Example - PDF File on Web - Multiple Pages
- if pdf has multiple tables, the output of `read_pdf` is a list of tables instead of just one DataFrame
- can check how many tables were extracted with len() and can also index the table to extract (e.g. `dfs[0]` for the first table)

In [19]:
url = 'https://members.rebgv.org/news/REBGV-Stats-Pkg-Mar-2023.pdf'
response = requests.get(url, proxies=proxies)
raw_pdf = response.content
dfs = read_pdf(io.BytesIO(raw_pdf), pages=3, multiple_tables=True)

In [20]:
len(dfs)

In [21]:
dfs[0].to_excel('Vancouver housing.xlsx')

In [23]:
dfs[0].head(10)

## Advanced Example - PDF with Custom Area
- if tabula cannot find the pdf, you can provide the location of the table with a top/left/height/width set of coordinates
- you can find these coordinates within Adobe with the Measure tool set to points (instead of inches)

In [36]:
url = "https://trreb.ca/files/market-stats/market-watch/mw2310.pdf"
response = requests.get(url, proxies=proxies)
raw_pdf = response.content
dfs = read_pdf(io.BytesIO(raw_pdf), pages=4, multiple_tables=True)

In [37]:
len(dfs)

In [41]:
dfs[0]
# dfs[1]

In [61]:
top = 76
left = 10.32
height = 577
width = 780.32
df = read_pdf(io.BytesIO(raw_pdf), pages=4, area=[top,left,height,width])

In [62]:
df

## Using Stream
- sometimes if the rows are unclear Tabula might do a poor job in finding the rows
- can use `lattice=True` or `stream=True`
- Lattice is used to parse tables that have demarcated lines between cells, while Stream is used to parse tables that have whitespaces between cells to simulate a table structure.

In [74]:
#https://panynj.gov/airports/en/statistics-general-info.html
url = 'https://panynj.gov/content/dam/airports/statistics/statistics-general-info/annual-atr/ATR_2022.pdf'

In [90]:
response = requests.get(url, proxies=proxies)
raw_pdf = response.content
df = read_pdf(io.BytesIO(raw_pdf), pages=23)

In [91]:
df

In [92]:
#Add stream argument
df = read_pdf(io.BytesIO(raw_pdf), pages=23, stream=True)

In [94]:
df

In [95]:
#need to break page into multiple tables
dfs = read_pdf(io.BytesIO(raw_pdf), pages=23, stream=True, multiple_tables=True)
len(dfs)

In [96]:
dfs[0]

In [97]:
dfs[1]